Andrea Palladio - Villa Badoer
================

Working again on Villa Badoer as started in workshop_05.
Aim of this final workshop is to completing the model.

Villa Badoer is a villa in Fratta Polesine in the Veneto region of northern Italy. It was designed in 1556 by Andrea Palladio for the Venetian noble Francesco Badoer, and built between 1557 and 1563, on the site of a medieval castle which guarded a bridge across a navigable canal. This was the first time Palladio used his fully developed temple pediment in the facade of a villa.
Villa Badoer has been part since 1996 of the UNESCO World Heritage Site "City of Vicenza and the Palladian Villas of the Veneto".

Constructed and inhabited in 1556, the villa therefore functioned for the management of the fields and was simultaneously a visible sign of the “feudal” presence, so to speak, of Badoer in the territory: it is not coincidental that the building rises on the site of an ancient medieval castle.
Palladio succeeded in uniting within one effective synthesis these dual meanings, joining the majestic manor house to the two barchesse (farm wings) bent into semicircles, which screen the stables and other agricultural annexes.

![VillaBadoer](./images/villa.jpg)

In [1]:
from larlib import *
from math import floor
import csv
import os

Evaluating fenvs.py..
...fenvs.py imported in 0.00617 seconds


# Reading data
As mentioned in workshop_05 I will assign 3d-points instead of getting data from the filesystem. This time I will put all data in a separate file, for readability.
This will prevent problems regarding paths etc. this approach may come in handy for fine tune points ;)

In [2]:
def larModel(path2lines, openPath=0):
    """
    larModel("mypath")
    :param path2file: path to the .lines file
    :return: tuple. A graph model representing Vertices and Edges.
             Edges are lists of indices to Vertices.
    """
    with open(path2lines, "rb") as file:
        lines = csv.reader(file, delimiter=",")

        mapping = {}
        v = []
        ev = []
        index = 0
        
        # indexing of the vertices and approximation
        for entry in lines:
            x1 = math.floor(float(entry[0])*10)/10
            y1 = math.floor(float(entry[1])*10)/10
            x2 = math.floor(float(entry[2])*10)/10
            y2 = math.floor(float(entry[3])*10)/10

            if not(mapping.has_key((x1,y1))):
                index += 1
                mapping[(x1,y1)] = index
                v.append([x1,y1])

            if not(mapping.has_key((x2,y2))):
                index += 1
                mapping[(x2,y2)] = index
                v.append([x2,y2])

            ev.append([mapping[(x1,y1)], mapping[(x2,y2)]])
        
        #this will close the path by linking the last node with the first one
        if (ev[-1][1] != ev[0][0] and not(openPath)):
            lastEdge = [ev[-1][1], ev[0][0]]
            ev.append(lastEdge)
        
        #print (lastEdge[0] == ev[-2][1]) #True
        #print (lastEdge[1] == ev[0][0]) #True
        #print ev[-1] == lastEdge
        
        
    return v,ev

# Work pipeline
- Divide et impera: I will start off with splitting the structure.
  - For every part of Villa Badoer a .csv and a .lines will be produced (https://inkscape.org)
  - Alternatively, if pre-existing pyplasm model for polyhedra are sufficient, then those will be used
- Given a .lines file for a certain part of the structure, produce a graph (V,EV) as lar model
- Given a part of the villa, draw its plant
- Given a plant use cartesian product to build walls etc.
- Figure out a way to use symmetries

In [3]:
from data import pyData # Pythonized data from .lines

# Base
Probably as a result of exploiting the substructures of the medieval castle, the manor house of the villa rises on a high basement, and recalls illustrious precedents like the Villa Medici at Poggio a Caiano by Giuliano da Sangallo, and the not far distant Villa dei Vescovi at Luvigliano by Giovanni Maria Falconetto.


The base on which the main structure lies is composed of two layers, base1 and base2, plus a frontal base adjacent to the stairs.
Base2 is placed upon base1, frontbase situates in front of base1 and base2 

In [4]:
#reload(pyData)

### Base 1

In [5]:
VBase1,EVBase1 = larModel(os.getcwd() + "/data/base1.lines")

In [6]:
VBase1 = pyData.VBase1
EVBase = pyData.EVBase1


In [7]:
VBase1

[[51.4, 83.4],
 [51.4, 120.7],
 [54.9, 120.7],
 [54.9, 115.7],
 [54.9, 115.7],
 [56.7, 115.7],
 [56.7, 120.7],
 [102.6, 120.7],
 [102.6, 115.6],
 [104.3, 115.6],
 [104.3, 120.7],
 [107.7, 120.7],
 [107.7, 83.4]]

### Base 2

In [8]:
VBase2,EVBase2 = larModel(os.getcwd() + "/data/base2.lines")

In [9]:
VBase2 = pyData.VBase2
EVBase2 = pyData.EVBase2

# Frontal base

In [10]:
VBaseF,EVBaseF = larModel(os.getcwd() + "/data/frontbase2.lines")

In [11]:
VBaseF = pyData.VBaseF
EVBaseF = pyData.EVBaseF

# Drawing the wire-frame plant

Again from workshop_05

In [12]:
def drawPlant(verts, edges, pr = 0):
    """
    drawPlant(V,EV) returns the wire-flame plant of the input, the lar model.
    It is clearly a 2d model, this implies that it does not handle space for doors, windows, stairs etc.
    
    :param verts: list of vertices in form [X, Y]
    :param edges: list of edges in form [indexofVertA, indexofVertB]
    :return: array of POLILINEs
    """
    plant = []
    for e in edges:
        if (pr):
            print ([verts[e[0]-1], verts[e[1]-1]]),"\n"
        plant.append(POLYLINE([verts[e[0]-1], verts[e[1]-1]]))
    return plant

In [13]:
base1plant = drawPlant(VBase1, EVBase1, pr=0)

In [14]:
base2plant = drawPlant(VBase2, EVBase2)


In [15]:
baseFplant = drawPlant(VBaseF, EVBaseF)
#VIEW(STRUCT(baseFplant))

# Building the bases

Scheme: First, I will build every part of the hole base (base1, base2 etc.) then I will use UNION to make a unique base out of each of its part.

Key functions:
- SOLIDIFY(polygon): it joins inner points of both concave and convex polygons
- UNION(polyhs): as the name suggets, it merges input polyhedra


I will assume steps have a height of 20cm and as we can see from the following image, the base has 9 + 5 steps

![steps](./images/gradini.jpg)

### Base 1

In [16]:
base1 = SOLIDIFY(STRUCT(base1plant))
base1 = PROD([base1, Q(0.20*9)])

In [17]:
#VIEW(base1)

### Base 2

In [18]:
base2 = SOLIDIFY(STRUCT(base2plant))
base2 = T(3)(0.20*9)(PROD([base2, Q(0.20*5)]))

In [19]:
#VIEW(base2)

### Frontal Base

In [20]:
baseF = SOLIDIFY(STRUCT(baseFplant))
baseF = PROD([baseF, Q(0.20*14)])

In [21]:
#VIEW(baseF)

## Final base with back stairs
Stairs can also be seen on the back of base1 and 2

In [22]:
def stairs(N,stepsize):
    """
    stairs build a basic, full stairway (no space between steps and the ground)
    :param N: number of steps
    :param stepsize: array of length 3. stepsize contains cartesian coordinate used to model a single step
    """
    sx,sy,sz = stepsize
    V,FV = larCuboids([1,1])
    step = S([1,2])([sx,sy])(STRUCT(MKPOLS((V,FV))))
    step =  steps = T(3)(sz*(N-1))(PROD([step,INTERVALS(sz)(1)]))
    for i in range(1,N):
        nextStep = T(3)(-sz*i)(S(2)(1+i)(step))
        steps = STRUCT([steps,nextStep])
    return steps

In [23]:
"""
Left-hand side back stairs

size depens on the svg/lines data.
It is redundant, in a way, to calculate it as a difference, but this way it gives the author more control.

I decided to model steps so that they starts from the origin [0,0,0], an approach I did not used for other structures.
This way, code will be more reusable. I will need to model various stairs.
"""
nStepsBack = 14 #recurrent value, as Villa Badoer has the same steps number for every stairway

bssX = 56.65 - 54.9
bssY = (117.7 - 115.7)/5.5
bssZ = 0.20 #height of a single step
backStepSize = [bssX, bssY, bssZ]

In [24]:
myStairs = stairs(nStepsBack, backStepSize)

## The whole base

- Base1
- Base2
- Frontal base
- Stairs on the back

In [25]:
bricksBase = './textures/bricks.jpeg'
orangeBricks = './textures/orangeBricks.jpg'

In [26]:
base1 = TEXTURE(bricksBase)(base1)
base2 = TEXTURE(bricksBase)(base2)
baseF = TEXTURE(orangeBricks)(baseF)

leftBackStairs = TEXTURE(bricksBase)(T([1,2])([54.93,115.7])(myStairs))
rightBackStairs = TEXTURE(bricksBase)(T([1,2])([102.55,115.60])(myStairs))

base = STRUCT([base1,base2, baseF, leftBackStairs, rightBackStairs])

In [27]:
#VIEW(base)

![base](./images/base.png)

![base_2](./images/base_2.png)

![base_3](./images/base_3.png)

# The main structure
Note: Some sources cite a basement beneth the main floor, yet no images or descriptions seem to be available. Because of that, I decided not to model a multiple floor structure until I find proper material on which to base a valid model.

## Floor
The floor lies on a base which is thinner then the ones we have seen before: it is slightly thinner than base2 (which is 0.20 \* 5), let's assume it is 0.2*3 m thick

In [28]:
VbaseVilla,EVbaseVilla = larModel(os.getcwd() + "/data/mainStrBase.lines", openPath=0)

In [29]:
VbaseVilla = pyData.VbaseVilla

In [30]:
EVbaseVilla = pyData.EVbaseVilla

In [31]:
#EVbaseVilla

In [32]:
villaBasePlant = drawPlant(VbaseVilla, EVbaseVilla)

""" total height of the base of the villa is approximately 10 steps, thus 0.20*10 """

villaBase1 = MULTEXTRUDE(OFFSET([1.1,1.1])(SOLIDIFY(STRUCT(villaBasePlant))))(0.20)
villaBase2 = MULTEXTRUDE(OFFSET([0.7,0.7])(STRUCT(villaBasePlant)))(0.20*8)
villaBase3 = MULTEXTRUDE(OFFSET([1.1,1.1])(SOLIDIFY(STRUCT(villaBasePlant))))(0.20)

villaBase = STRUCT([TOP([TOP([villaBase1, villaBase2]), villaBase3])])

In [33]:
#VIEW(villaBase)

In [34]:
villaFloor = MULTEXTRUDE(OFFSET([0.7,0.7])(SOLIDIFY(STRUCT(villaBasePlant))))(0.06)

## Inner walls.
I would like to use something different from the boolean operation DIFFERENCE to create space for doors. I will stack two model one on top of the other, the former one contains spaces, the latter one does not.

In [35]:
VinnWalls,EVinnWalls = larModel(os.getcwd() + "/data/innerwalls.lines", openPath=1)

In [36]:
VinnWallsTOP,EVinnWallsTOP = larModel(os.getcwd() + "/data/innerwallsTOP.lines", openPath=1)

In [37]:
VinnWallsTOP = pyData.VinnWallsTOP

In [38]:
EVinnWallsTOP = pyData.EVinnWallsTOP

In [39]:
innWallsPlantHoled = STRUCT(drawPlant(VinnWalls, EVinnWalls))
innWallsPlantHoled = MULTEXTRUDE(OFFSET([0.7,0.7])(STRUCT([innWallsPlantHoled])))(2.5)

In [40]:
innWallsPlantTOP = STRUCT(drawPlant(VinnWallsTOP, EVinnWallsTOP))
innWallsPlantTOP = MULTEXTRUDE(OFFSET([0.7,0.7])(STRUCT([innWallsPlantTOP])))(5.5)

In [41]:
innWalls = STRUCT([TOP([innWallsPlantHoled, innWallsPlantTOP])])
#VIEW(innWalls)

In [42]:
travertineHEX = "#E2DECD"

In [43]:
innWalls = HEX(travertineHEX)(innWalls)
#VIEW(innWalls)

![walls1](./images/walls.png)

![walls2](./images/walls2.png)

![walls3](./images/walls3.png)

In [44]:
parquet = './textures/parquet.jpg'

# Inner walls, floor and base
Let's now give a look inside the villa at current state 

In [45]:
villaBase = HEX(travertineHEX)(villaBase)
innWalls = HEX(travertineHEX)((innWalls))
villaFloor = TEXTURE(parquet)(villaFloor)
habitation = STRUCT([TOP([TOP([villaBase, villaFloor]), innWalls])])
#VIEW(habitation)

![inside](./images/inside.png)

![inside2](./images/inside2.png)

In [46]:
#VIEW(TOP([base, habitation]))

In [47]:
def RELATIVE(ref, obj, x=0, y=0, z=0):
    _x = float(x)
    _y = float(y)
    _z = float(z)
    return ALIGN([[1, MED, MED], [2, MED, MED], [3, MIN, MIN]])([
        ref,
        STRUCT([EMPTY, T([1, 2, 3])([-_x*2, -_y*2, _z*2])(obj)])
    ])

# Windows
Few kinds of windows:
- lower ones are placed on villaBase.
- big green windows are more "central" on walls height
- small green windows are place at the very top of the side walls

![side](./images/sideVillaFoto.jpg)

In [48]:
woodHEX = "#8B5A2B"
steelGreyHEX = "#43464B"
glassTexture = './textures/glass.jpeg'
greenWindowTexture1 = './textures/greenWindow1.png'
greenWindowTexture2 = './textures/greenWindow2.png'

### lower windows

In [49]:
def lowerWindow():
    windowStructure = SKEL_1(STRUCT([PROD([INTERVALS(0.20*8)(2), INTERVALS(0.20*5)(1)])]))
    windowStructure = HEX(woodHEX)(STRUCT([OFFSET([0.01,0.03, 0.03])(PROD([Q(0.01), windowStructure]))]))

    #SIZE([1,2,3])(windowStructure) = [0.019999999552965164, 1.6299999952316284, 1.0299999713897705]
    glassSize = [0.019999999552965164/4, 1.6299999952316284, 1.0299999713897705]
    glass = TEXTURE(glassTexture)(CUBOID(glassSize))
    
    windowStructureBack = CUBOID(glassSize)
    windowStructureBack = HEX(woodHEX)(windowStructureBack)
    
    windowGrid = SKEL_1(STRUCT([PROD([INTERVALS(1.62)(6), INTERVALS(1.01)(4)])]))
    windowGrid = HEX(steelGreyHEX)(STRUCT([OFFSET([0.01,0.01, 0.01])(PROD([Q(0.01), windowGrid]))]))

    window = STRUCT([RIGHT([RIGHT([windowStructureBack, RIGHT([glass, windowStructure])]), windowGrid])])
    
    return window

In [50]:
myLowerWindow = lowerWindow()
#VIEW(myLowerWindow)

In [51]:
def moveRelatively(ref, obj, x=0, y=0, z=0):
    _x = float(x)
    _y = float(y)
    _z = float(z)
    return ALIGN([[1, MED, MED], [2, MED, MED], [3, MIN, MIN]])([
        ref, STRUCT([EMPTY, T([1, 2, 3])([_x*2, -_y*2, _z*2])(obj)])
    ])

### big green windows

In [52]:
def greenWindow(n,rot=1.0):
    greenWindowSize = [0.019999999552965164, 1.6299999952316284, n*1.0299999713897705]
    window = TEXTURE([greenWindowTexture1, True,True, 0.0,0.0,0.0,1.0,1.0*rot,0.0,0.0])(CUBOID(greenWindowSize))
    return window

In [53]:
myBigGrWindow = greenWindow(3)
#VIEW(myBigGrWindow)

### small green windows

In [54]:
myLittleGrWindow = greenWindow(1,rot=PI)
#VIEW(myLittleGrWindow)

In [55]:
def placeWindows():
    """
    This method aims to properly arrange windows on the vertical axis.
        - small green window ()
        - big green window
        - lower window
    :return: pyplasm.xgepy.Hpc
    """
    #the list of all the windows
    windowStack = []
    
    lowerWin = T(3)(0.20*5)(lowerWindow())
    bigGreen = T(3)(0.20*15)(greenWindow(3))
    smallGreen = T(3)(5.)(greenWindow(1,rot=PI))
    
    windowStack.append(lowerWin)
    windowStack.append(bigGreen)
    windowStack.append(smallGreen)
    
    return STRUCT(windowStack)

In [56]:
myWindowStack = placeWindows()
#VIEW(myWindowStack)

## Outer Walls - façades
Or the walls of the villa, these will host the windows.
Side walls are (95.9 - 58.9) long, this value is obtained from svg taken for the floor and villaBase.

Let's remember window sizes: 
    [0.019999999552965164, 1.6299999952316284, 1.0299999713897705] = [thickness, width, height]  
    with n = 4.5 for the big green windows and n = 1 for the smaller ones

In [57]:
def sideFacade():
    sideWallsWidth = 95.9 - 58.9
    windowSizes = [0.019999999552965164, 1.6299999952316284, 1.0299999713897705]

    # QUOTE uses an array of integers: positive ones will be segments, negative ones spaces.
    # fullWall<D> is a positive integer used in QUOTE to represent segments, or portions of the full wall
    # (no windows)
    fullX = sideWallsWidth - windowSizes[1]*3.
    fullWallX = fullX/4. - windowSizes[1]/2
    
    # some datapoint are summed up with long values,
    # this has the mere aim of reaching the proper width of the wall. See below prints
    xRithm = [fullWallX, -windowSizes[1], fullWallX+1.3039999961999995, -windowSizes[1],
              fullWallX+1.9559999942999995, -windowSizes[1], fullWallX]
    
    # just to be shure that if we sum up these elements, we get the actual width/height of the wall
    print "sideWallsWidth: {}".format(sideWallsWidth)
    print ("sumFullWallX: {}".format(sum([abs(n) for n in xRithm])))
    
    
   
    y =  windowSizes[2]   
    yRithm = [0.4, -y*3., y*3, -y, 0.39000020027000026]
    print ("wall height: 8.0")
    print ("sumFullWallY: {}".format(sum([abs(n) for n in yRithm])))
    
    xVoid = [-xv for xv in xRithm]
    yVoid = [-yv for yv in yRithm]
    
    return STRUCT([
        PROD([QUOTE(xRithm),QUOTE(yRithm)]),
        PROD([QUOTE(xVoid),QUOTE(yRithm)]),
        PROD([QUOTE(xRithm),QUOTE(yVoid)])
        ])

In [58]:
myFacade = sideFacade()

sideWallsWidth: 37.0
sumFullWallX: 37.0
wall height: 8.0
sumFullWallY: 8.0


In [59]:
#VIEW(myFacade)

![sideQuote](./images/sideQuote.png)

## Building the back wall
Here we have a more intricate, structure for the back wall. It is possible to look at this wall as a structure divided in three parts:
- a central part, containing 3 big green windows close to one another
- left part and right side are specular

I made some assumption to give a proportion to each part (I could have measured pixels with inkScape but it feels excessive), given the width of 41.4 : 
- cental part = $\frac{1}{4}\cdot 41.4$ = 10.35
- left/right parts (I will call this lrSpace, now on) = $\frac{3}{8}\cdot 41.4$ (each) = 15.524999999999999 (each)

![back](./images/back.jpeg)

In [60]:
def SPECULAR (D):
    """
    SPECULAR(D)(pol) is just a variation of MIRROR(D)(pol) without returning the original object,
    plus the specular polyhedron is placed so that its origin equals pol origin
    :param D: direction, integer
    :param pol: polyhedron
    """
    def SPECULAR0 (pol):
        return  STRUCT([T(D)(SIZE(D)(pol))(S(D)(-1)(pol))])
    return SPECULAR0

### Important Note
One single jupyter does not seem to be able to handle some kind of computation (the kernel dies), as the one for the back wall i originally intended to use.  
I need to split the creation of the back walls in more than one method, possible scheme:
    - part 1
    - part 2
    - part 3
    - ...
    
Below the central part of this wall is divided in three parts, each one is associated with a bigGreenWindows plus grouping the creation of all these part in a unique method, as mentioned above is not possible within a Jupyter Notebook

![back2](./images/back2.jpeg)

### façade
I will now use a support method to build a model for a façade given the geometric pattern representing it (e.g. window-no window-window ...)

This method has been inspired from PLASM documentation (chapter 02)

In [61]:
def facade(args):
    xRithm, yRithm, xVoid, yVoid = args
    
    wall = STRUCT([
    PROD([QUOTE(xRithm),QUOTE(yRithm)]),
    PROD([QUOTE(xVoid),QUOTE(yRithm)]),
    PROD([QUOTE(xRithm),QUOTE(yVoid)])
    ])
    return wall

In [62]:
windowSizes = [0.019999999552965164, 1.6299999952316284, 1.0299999713897705]
wx = windowSizes[1]
wy =  windowSizes[2]

In [63]:
def backWallSide(x, y):
    """
    backWallSide build one side of the wall, it will just be necessary to apply SPECULAR to it, to get the other side
    :param x: horizontal measure of the basic small window
    :param y: vertical measure of the basic small window
    :return: pyplasm.xgepy.Hpc
    """
    lrSpaceX = 15.525 #it will be automatically rounded to this value (no need to to use 15.5249...)
        
    xRithm = [lrSpaceX/3.-0.175, -x, lrSpaceX/3.+0.4, -x, 1.6900000095000003]  
    yRithm = [0.4, -y*3., y*3, -y, 0.39000020027000026]
    xVoid = [-xv for xv in xRithm]
    yVoid = [-yv for yv in yRithm]
    
    return facade([xRithm, yRithm, xVoid, yVoid])

In [64]:
backWallLeft = backWallSide(wx, wy)
backWallRight = SPECULAR(1)(backWallLeft)

#VIEW(STRUCT([backWallLeft, T(1)(15.525)(backWallRight)]))

In [65]:
def backWallCentralSides(x, y):
    """
    This creates the lateral side of the central part
    :param x: horizontal measure of the basic small window
    :param y: vertical measure of the basic small window
    :return: list cointaing both sides
    """
    out = []
    
    xRithm = [1.91000000834465, -x, 2.]
    yRithm = [0.4, -y*12./5, y*18./5, -y, 0.39000020027000026]
    xVoid = [-xv for xv in xRithm]
    yVoid = [-xv for xv in yRithm]
    facade1 = facade([xRithm, yRithm, xVoid, yVoid])
    out.append(facade1)
       
    out.append(SPECULAR(1)(facade1))
    return out

In [66]:
"""
IL PROBLEMA È QUI
"""

def backWallCentralMain(x, y):
    
    yRithm = [0.1,-y*3, 4.9100000858307]
    #xVoid = [-xv for xv in xRithm]
    yVoid = [-xv for xv in yRithm]
    
    centralMain = STRUCT([
    PROD([Q(x),QUOTE(yRithm)]),
    PROD([Q(x),QUOTE(yVoid)])
    ])
    return centralMain

In [67]:
#AA(VIEW)(backWallCentralSides(wx, wy))

In [68]:
b = backWallCentralMain(wx, wy)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181c191c00> >